# Ray Crash Course - Running Ray Clusters

How to run a Ray Cluster to scale beyond a single machine. How to submit work to the cluster and monitor progress. How to integrate cluster execution with your local development environment.

TODO

In [ ]:
!../tools/start-ray.sh

In [ ]:
ra

In [ ]:
ray.shutdown()  # "Undo ray.init()". Terminate all the processes started in this notebook.